# Spotify Authentication Process

This guide is based off of the Spotify authentication documentation.

## SETUP
The first step is to register an app. This can be done from the Spotify Developer dashboard: https://developer.spotify.com/dashboard/applications

When you setup your application, Spotify will ask you for a "Redirect URI" during the registration process. For the purposes of this tutorial, "http://127.0.0.1/" is used.

After registering an app you will recieve a "Client ID" and a "Client Secret". For the purpose of this tutorial, both are set as Environment Variables but feel free to replace the variables below to plain text if security is not an issue.

In [ ]:
# Run this code FIRST to import libraries and set Environment Variables

import requests
import json

client_id = os.getenv("SPOTIFY_CLIENT_ID")
client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")
redirect_uri = "http://127.0.0.1/"

## 1. Have your application request authorization; the user logs in and authorizes access

This is the first step in the Spotify auth process. This will generate a URL will be used to get the authentication code confirming that the user is giving the app permission to access their account.

In [ ]:
def get_spotify_auth_url():
    payload = {
        "client_id":client_id,
        "response_type": "code",
        "redirect_uri": redirect_uri,
        "scope": "playlist-modify-private playlist-modify-public"
        }
    auth_url = requests.get("https://accounts.spotify.com/authorize", params=payload).url

    # returns an auth_url which is used to get auth_code
    return auth_url

spotify_auth_url = get_spotify_auth_url()
print("Please go to the following URL. You will be redirrected to a new URL, please paste the COMPLETE URL in the following dialog box")
print(spotify_auth_url)
auth_code = input()
auth_code = auth_code.split("code=")[1]

print(auth_code)

## 2. Have your application request refresh and access tokens; Spotify returns access and refresh tokens
This takes the Spotify auth_code and returns an access_token which can be used to make Spotify API requests.
The refresh_token can be used to get a new access_token at a later time (typically expires in one hour).

In [ ]:
def get_spotify_access_token(auth_code):
    data = {
        "client_id": client_id,
        "client_secret": client_secret,
        "grant_type": "authorization_code",
        "code": auth_code,
        "redirect_uri": redirect_uri
        }
    resp = requests.post("https://accounts.spotify.com/api/token", data = data).content
    access_token_resp = json.loads(resp)

    # returns dictionary which contains access_token and refresh_token, among other things
    return access_token_resp

spotify_access_token = get_spotify_access_token(auth_code)

access_token = spotify_access_token.get("access_token")
refresh_token = spotify_access_token.get("refresh_token")

## 3. Use refresh token to generate new auth_tokens
access_token will expire after a while (60 minutes at the time of this guide). Once they expire they cannot be used. If the access_token has expired you can use the refresh_token to generate a new access_token

In [ ]:
def spotify_refresh_token(refresh_token):
    data = {
        "client_id": client_id,
        "client_secret": client_secret,
        "grant_type":"refresh_token",
        "refresh_token":refresh_token
        }
    resp = requests.post("https://accounts.spotify.com/api/token", data = data).content
    access_token_resp = json.loads(resp)
    access_token = access_token_resp.get("access_token")
    return access_token

access_token = spotify_refresh_token(refresh_token)
print(access_token)